In [1]:
from pynq import Overlay
import numpy as np
import random
from pynq import allocate

overlay = Overlay('/home/xilinx/pynq/overlays/aes/aes_axis.bit')
overlay?

In [2]:
aes_ip = overlay.aes_0
aes_ip?

In [ ]:
dma = overlay.axi_dma_0
dma_send = overlay.axi_dma.sendchannel
dma_recv = overlay.axi_dma.recvchannel

In [ ]:
aes_ip.register_map

In [ ]:
CONTROL_REGISTER = 0x0
aes_ip.write(CONTROL_REGISTER, 0x01)

In [3]:
aes_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  plaintext_1 = Register(plaintext=write-only),
  plaintext_2 = Register(plaintext=write-only),
  ciphertext_1 = Register(ciphertext=write-only),
  ciphertext_2 = Register(ciphertext=write-only),
  key_1 = Register(key=write-only),
  key_2 = Register(key=write-only),
  key_size = Register(key_size=write-only),
  decryptedtext_1 = Register(decryptedtext=write-only),
  decryptedtext_2 = Register(decryptedtext=write-only)
}

In [4]:
data_size = 16

key_buffer = allocate(shape=(data_size,), dtype=np.uint32)
key_array = [ord(char) for char in "kkkkeeeeyyyy...."]
key_buffer[:] = key_array

plaintext_buffer = allocate(shape=(data_size,), dtype=np.uint32)
plaintext_array = [ord(char) for char in "abcdef1234567890"]
plaintext_buffer[:] = plaintext_array

ciphertext_buffer = allocate(shape=(data_size,), dtype=np.uint32)
decrypted_plaintext_buffer = allocate(shape=(data_size,), dtype=np.uint32)

In [ ]:
aes_ip.write(0x34, 16) # Write Key Size = 16
dma_send.transfer(key_buffer)
dma_send.transfer(plaintext_buffer)

dma_recv.transfer(ciphertext_buffer)
dma_recv.transfer(decrypted_plaintext_buffer)

dma_send.wait()
dma_recv.wait()


In [5]:
ciphertext_buffer

In [6]:
decrypted_plaintext_buffer

In [10]:
del key_buffer
del plaintext_buffer
del ciphertext_buffer
del decrypted_plaintext_buffer